In [ ]:
import numpy as np
import time as time
import os
import constants

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

# Define cosmology
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)


PiBound = (min(PiEdges), max(PiEdges) )

# Read in galaxies to provide counts in plot titles
galfil = os.path.join(constants.GAL_DIR_BASE, 'cat_galxcorr_cl2020_nonuniq_v0.dat')
gal = ascii.read(galfil,format='ipac')
#print(gal.columns)

gal_3d = gal[gal['source'] == '3DHST']
gal_zD = gal[gal['source'] == 'zDeep']
gal_mosdef = gal[gal['source'] == 'MOSDEF']
gal_vuds = gal[gal['source']=='VUDS']
gal_clamato=gal[gal['source']=='CLAMATO']

In [ ]:
OBS_DIR = os.path.join(constants.XCORR_DIR_BASE, 'obs')

XCorr_3d=np.load(os.path.join(OBS_DIR, f"xcorr_3dhst_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_3d = np.transpose(XCorr_3d)
    
XCorr_zD = np.load(os.path.join(OBS_DIR, f"xcorr_zDeep_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_zD = np.transpose(XCorr_zD)
    
XCorr_mosdef = np.load(os.path.join(OBS_DIR, f"xcorr_mosdef_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_mosdef = np.transpose(XCorr_mosdef)
    
XCorr_vuds = np.load(os.path.join(OBS_DIR, f"xcorr_vuds_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_vuds = np.transpose(XCorr_vuds)

XCorr_clamato = np.load(os.path.join(OBS_DIR, f"xcorr_clamato_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_clamato = np.transpose(XCorr_clamato)

XCorr_all = np.load(os.path.join(OBS_DIR, f"xcorr_all_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_all = np.transpose(XCorr_all)

In [ ]:
X, Y = np.meshgrid(SigEdges, PiEdges)

SigMax = 35.
PiMin = -40.
PiMax = 40.

fig, ((ax1, ax2, ax3),(ax4,ax5,ax6))= plt.subplots(2,3, figsize=(12,14))

print(len(X), len(Y), np.shape(XCorrPlot_mosdef))

ax1.pcolormesh(X, Y, XCorrPlot_mosdef,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax1.set_aspect('equal')
ax1.set_xlim(np.min(X), SigMax)
ax1.set_ylim(PiMin, PiMax)
ax1.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax1.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax1.set_title("MOSDEF: %i galaxies" % len(gal_mosdef))
ax1.axhline(-1.01 / cosmo.h, color='black', label='Diag covar fit')
ax1.axhline(0 / cosmo.h, color='black', ls='--', label='Full covar fit')

ax2.pcolormesh(X, Y, XCorrPlot_vuds,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax2.set_aspect('equal')
ax2.set_xlim(np.min(X), SigMax)
ax2.set_ylim(PiMin, PiMax)
ax2.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax2.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax2.set_title("VUDS: %i galaxies" % len(gal_vuds))
# ax2.axhline(1.15 / cosmo.h, color='black', label='Diag covar fit')
# ax2.axhline(0.395 / cosmo.h, color='black', ls='--', label='Full covar fit')
# ax2.axhline(1.75 / cosmo.h, color='purple', label='Chi-by-eye')


ax3.pcolormesh(X, Y, XCorrPlot_zD,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax3.set_aspect('equal')
ax3.set_xlim(np.min(X), SigMax)
ax3.set_ylim(PiMin, PiMax)
ax3.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax3.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax3.set_title("zCOSMOS: %i galaxies" % len(gal_zD))
ax3.axhline(2.75 / cosmo.h, color='black', label='Diag covar fit')
ax3.axhline(2.75 / cosmo.h, color='black', ls='--', label='Full covar fit')

pcm=ax4.pcolormesh(X, Y, XCorrPlot_3d,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax4.set_aspect('equal')
ax4.set_xlim(np.min(X), SigMax)
ax4.set_ylim(PiMin, PiMax)
ax4.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax4.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax4.set_title("3DHST: %i galaxies" % len(gal_3d))
ax4.axhline(-3.2 / cosmo.h, color='black', label='Diag covar fit')
ax4.axhline(-1.3 / cosmo.h, color='black', ls='--', label='Full covar fit')

pcm=ax5.pcolormesh(X, Y, XCorrPlot_clamato,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax5.set_aspect('equal')
ax5.set_xlim(np.min(X), SigMax)
ax5.set_ylim(PiMin, PiMax)
ax5.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax5.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax5.set_title("CLAMATO: %i galaxies" % len(gal_clamato))
ax5.axhline(2.1 / cosmo.h, color='black', label='Diag covar fit')
ax5.axhline(1.6 / cosmo.h, color='black', ls='--', label='Full covar fit')

pcm=ax6.pcolormesh(X, Y, XCorrPlot_all,cmap='jet_r',vmin=-0.2, vmax=0.1 )
ax6.set_aspect('equal')
ax6.set_xlim(np.min(X), SigMax)
ax6.set_ylim(PiMin, PiMax)
ax6.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
ax6.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
ax6.set_title("All: %i galaxies" % len(gal))

ax6pos = ax6.get_position()
cbaxes = fig.add_axes([ax6pos.x0+0.04, ax6pos.y0+0.25, ax6pos.width, ax6pos.height*0.8])
cbaxes.axis('off')
fig.colorbar(pcm, ax=cbaxes)

#fig.subplots_adjust(wspace=-0.1)

plt.show()

In [ ]:
pi_flat_plot = lambda xcorr, lab: plt.plot(PiEdges[1:], np.sum(xcorr, axis=1), label=lab)

pi_flat_plot(XCorrPlot_mosdef, 'MOSDEF')
pi_flat_plot(XCorrPlot_vuds, 'VUDS')
pi_flat_plot(XCorrPlot_zD, 'zCOSMOS')
pi_flat_plot(XCorrPlot_3d, '3DHST')
pi_flat_plot(XCorrPlot_clamato, 'CLAMATO')
plt.xlabel(r'$\pi\; (\mathrm{cMpc})$')
plt.ylim(-1.5, 0.3)
plt.legend()

In [ ]:
sigma_flat_plot = lambda xcorr, lab: plt.plot(SigEdges[1:], np.sum(xcorr, axis=0), label=lab)

sigma_flat_plot(XCorrPlot_mosdef, 'MOSDEF')
sigma_flat_plot(XCorrPlot_vuds, 'VUDS')
sigma_flat_plot(XCorrPlot_zD, 'zCOSMOS')
sigma_flat_plot(XCorrPlot_3d, '3DHST')
sigma_flat_plot(XCorrPlot_clamato, 'CLAMATO')
plt.xlabel(r'$\sigma\; (\mathrm{cMpc})$')
plt.xlim(0, 35)
plt.ylim(-5, 0.5)
plt.legend()